# Abdullah Alhussni - aa10108
## Applied Machine Learning - ENGR-UH 3332 - Project 1

### 1 + 2 - Importing the data set and setting up functions

In [206]:
from sklearn.datasets import load_boston
import numpy as np
import matplotlib.pyplot as plt
import warnings
from sklearn.model_selection import KFold
from sklearn.preprocessing import PolynomialFeatures

## Avoid printing out warnings

with warnings.catch_warnings():
     warnings.filterwarnings("ignore")
     X, y = load_boston(return_X_y=True)

a = 2
r = 0.1

def gradient(X, y, y_pred, theta):
	return (
		- (2 / y.size) * X.T.dot(y - y_pred) # gradient of mse
	)

def lasso_gradient(X, y, y_pred, theta):
	return (
		- (2 / y.size) * X.T.dot(y - y_pred) # gradient of mse
		+ a * np.sign(theta)		         # subgradient of 11-penalty
	)

def elastic_gradient(X, y, y_pred, theta):
	return (
		- (2 / y.size) * X.T.dot(y - y_pred) # gradient of mse
		+ a * np.sign(theta)		         # subgradient of 11-penalty
        + a * (1 - r) * theta 			     # gradient of 12-penalty
	)

def find_theta(X, y, l): # for both regular and ridge regression
    m = X.shape[0]
    n = X.shape[1]
    a = np.append(X, np.ones((m, 1)), axis = 1)
    return np.dot(np.linalg.pinv(np.dot(X.T, X) + l * np.identity(n)), np.dot(X.T, y))

def predict(X, theta):
    m = X.shape[0]
    a = np.append(X, np.ones((m, 1)), axis = 1)
    return np.dot(X, theta)

### 3 - Linear regression, closed form

In [207]:
theta3 = find_theta(X, y, 0)
y_pred3 = predict(X, theta3)

k = 7

kf = KFold(n_splits = k)

kf3 = np.column_stack((X, y))

scores3 = []
for train, test in kf.split(kf3):
    X_train = kf3[train, :-1]
    y_train = kf3[train, -1]
    X_test = kf3[test, :-1]
    y_test = kf3[test, -1]

    theta_temp = find_theta(X_train, y_train, 0)
    y_pred_temp = predict(X_test, theta_temp)
    diff = y_pred_temp - y_test
    MSE = sum(diff ** 2) / diff.size
    scores3.append(MSE)

print(np.average(scores3))

34.32705988271211


### 4 - Ridge regression, closed form

In [208]:
lambdas = np.logspace(1, 7, num = 13)

theta4 = np.zeros((X.shape[0], lambdas.size))
y_pred4 = np.zeros((X.shape[1], lambdas.size))

scores4 = np.zeros((k, lambdas.size))

for i in range(len(lambdas)):
    for j, (train, test) in enumerate(kf.split(kf3)):
        X_train = kf3[train, :-1]
        y_train = kf3[train, -1]
        X_test = kf3[test, :-1]
        y_test = kf3[test, -1]
    
        theta_temp = find_theta(X_train, y_train, lambdas[i])
        y_pred_temp = predict(X_test, theta_temp)
        diff = y_pred_temp - y_test
        MSE = sum(diff ** 2) / diff.size
        scores4[j, i] = MSE
    print(np.average(scores4[:, i]))

33.8710292667754
33.657553485206826
34.857971150261946
41.34735547311392
51.95142222059484
60.41843532305361
67.55922935851414
74.76694253933445
80.3821378200417
84.63246275865025
88.13311797809762
93.46530228333746
109.26108023258048


As we can see, lambda = 10^1.5 = 10sqrt(10) = 31.6 gives the lowest MSE (in fact, most values between 1 and 100 give similar MSE, hence lambda = 10 is also a safe choice) and hence the least error. Trying lambda < 10 does not give better results, as MSE does not seem to go below 33.6.

### 5 - Ridge regression, optimal lambda

In [209]:
lambda5 = 10 ** 1.5

theta5 = find_theta(X, y, lambda5)
y_pred5 = predict(X, theta5)

scores5 = []
for train, test in kf.split(kf3):
    X_train = kf3[train, :-1]
    y_train = kf3[train, -1]
    X_test = kf3[test, :-1]
    y_test = kf3[test, -1]

    theta_temp = find_theta(X_train, y_train, lambda5)
    y_pred_temp = predict(X_test, theta_temp)
    diff = y_pred_temp - y_test
    MSE = sum(diff ** 2) / diff.size
    scores5.append(MSE)

print(np.average(scores5))

33.657553485206826


### 6 - Polynomial regression

In [210]:
poly = PolynomialFeatures(degree = 2)
X_poly = poly.fit_transform(X)

kf6 = np.column_stack((X_poly, y))

theta6 = np.zeros((X_poly.shape[1], lambdas.size))
y_pred6 = np.zeros((X_poly.shape[0], lambdas.size))

scores6 = np.zeros((k, lambdas.size))

for i in range(len(lambdas)):
    theta6[:, i] = find_theta(X_poly, y, lambdas[i])
    y_pred6[:, i] = predict(X_poly, theta6[:, i])
    
    for j, (train, test) in enumerate(kf.split(kf6)):
        X_train = kf6[list(train), :-1]
        y_train = kf6[list(train), -1]
        X_test = kf6[list(test), :-1]
        y_test = kf6[list(test), -1]

        theta_temp = find_theta(X_train, y_train, lambdas[i])
        y_pred = predict(X_test, theta_temp)
        diff = y_test - y_pred
        MSE = np.sum(diff ** 2) / diff.size
        scores6[j, i] = MSE
    
    print(np.average(scores6[:, i]))

80.32452022742493
59.63938477118783
47.871789932177414
42.25510311460955
35.303492509184835
28.610349447086495
28.156289582154177
30.15353681290134
32.71503922653546
37.32136948595105
42.595151451910674
46.53114246921746
46.384473402275276


Here the lowest MSE is going as low as 28.1 for lambda = 10000.

### 7 - Gradient descent, gradient of MSE only

In [211]:
eta = 0.000001
n_iterations = 100000

theta7 = np.random.randn(X.shape[1])

for iteration in range(n_iterations):
    y_pred7 = predict(X, theta7)
    grad = gradient(X, y, y_pred7, theta7)
    theta7 = theta7 - eta * grad

scores7 = []
for train, test in kf.split(kf3):
    X_train = kf3[train, :-1]
    y_train = kf3[train, -1]
    X_test = kf3[test, :-1]
    y_test = kf3[test, -1]
    
    theta_temp = np.random.randn(X.shape[1])
    for iteration in range(n_iterations):
        y_pred_temp = predict(X_train, theta_temp)
        grad_temp = gradient(X_train, y_train, y_pred_temp, theta_temp)
        theta_temp = theta_temp - eta * grad_temp

    y_pred_temp = predict(X_test, theta_temp)
    diff = y_pred_temp - y_test
    MSE = sum(diff ** 2) / diff.size
    scores7.append(MSE)

print(np.average(scores7))

88.18537597878502


After experimenting with the learning rate and n_iterations values, I settled on these values as they give the smallest error (relative to the linear regression closed-form solution). Still, this method was quite unstable, partially due to the random vector assumption but mostly due to the difficulty of picking a specific learning rate or maximum number of iterations with no mathematical equations helping.

### 8 - Gradient descent, lasso gradient (additional l1-penalty term)

In [212]:
theta8 = np.random.randn(X.shape[1])

for iteration in range(n_iterations):
    y_pred8 = predict(X, theta8)
    grad = lasso_gradient(X, y, y_pred8, theta8)
    theta8 = theta8 - eta * grad

# print(theta3)
# print(theta7 - theta3)

scores8 = []
for train, test in kf.split(kf3):
    X_train = kf3[train, :-1]
    y_train = kf3[train, -1]
    X_test = kf3[test, :-1]
    y_test = kf3[test, -1]
    
    theta_temp = np.random.randn(X.shape[1])
    for iteration in range(n_iterations):
        y_pred_temp = predict(X_train, theta_temp)
        grad_temp = lasso_gradient(X_train, y_train, y_pred_temp, theta_temp)
        theta_temp = theta_temp - eta * grad_temp

    y_pred_temp = predict(X_test, theta_temp)
    diff = y_pred_temp - y_test
    MSE = sum(diff ** 2) / diff.size
    scores8.append(MSE)

print(np.average(scores8))

52.822257157647805


### 9 - Gradient descent, elastic gradient (additional l1-penalty and l2-penalty terms)

In [213]:
theta9 = np.random.randn(X.shape[1])

for iteration in range(n_iterations):
    y_pred9 = predict(X, theta8)
    grad = elastic_gradient(X, y, y_pred9, theta9)
    theta9 = theta9 - eta * grad

scores9 = []
for train, test in kf.split(kf3):
    X_train = kf3[train, :-1]
    y_train = kf3[train, -1]
    X_test = kf3[test, :-1]
    y_test = kf3[test, -1]
    
    theta_temp = np.random.randn(X.shape[1])
    for iteration in range(n_iterations):
        y_pred_temp = predict(X_train, theta_temp)
        grad_temp = elastic_gradient(X_train, y_train, y_pred_temp, theta_temp)
        theta_temp = theta_temp - eta * grad_temp

    y_pred_temp = predict(X_test, theta_temp)
    diff = y_pred_temp - y_test
    MSE = sum(diff ** 2) / diff.size
    scores9.append(MSE)

print(np.average(scores9))

104.2775649650509


### 10

After comparing the resulting MSE from each model, I see that polynomial regression was the most successful comparatively. Other than avoiding randomness that does not feature much mathematical rigor (so far), it also provided the least MSE values (although scaling the data could have potentially produced different results). Also, when comparing ridge, lasso, and elastic gradient, they seemed to jump around a lot. I believe with better initial guesses, learning rate, maximum number of iterations, and coefficients for lasso and elastic regression, all gradient descent have better potential, but it just happened that my experiment was a bit better with quadratic regression.